# Importing Liberaries

In [ ]:
!pip install mediapipe

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 32.8 MB 1.3 MB/s 


In [ ]:
import scipy.io
import numpy as np
import pandas as pd
import math
import glob
import os
import cv2
import mediapipe as mp
import numpy as np
import time
import pandas as pd
import os


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(min_detection_confidence=0.5, min_tracking_confidence=0.5)
mp_drawing = mp.solutions.drawing_utils
drawing_spec = mp_drawing.DrawingSpec(thickness=1, circle_radius=1)
imgnolist=[]

# creating empty dataframe to append points on it 
dfx=pd.DataFrame()
dfy=pd.DataFrame()


path='/content/drive/MyDrive/Head Pose Estimation ML Project/Dataset/Images'

for img in glob.glob(path+'/*.jpg'):
   
    image=cv2.imread(img,0)      
    # Flip the image horizontally for a later selfie-view display
    # Also convert the color space from BGR to RGB      
    image = cv2.cvtColor(cv2.flip(image, 1), cv2.COLOR_BGR2RGB)
    # To improve performance
    image.flags.writeable = False
        
    # Get the result
    results = face_mesh.process(image)     
    # To improve performance
    image.flags.writeable = True     
    # Convert the color space from RGB to BGR
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

    img_h, img_w, img_c = image.shape
    x_points=[]
    y_points=[]
    face_2d = []
    
    if results.multi_face_landmarks:
        #1. save mat file no
        img_no, _ = os.path.splitext(os.path.basename(img))
        imgnolist.append(img_no)
        
        for face_landmarks in results.multi_face_landmarks:
            for idx, lm in enumerate(face_landmarks.landmark):
                # if idx == 33 or idx == 263 or idx == 1 or idx == 61 or idx == 291 or idx == 199 :
                    if idx == 1: nose_2d = (lm.x * img_w, lm.y * img_h)
                    x, y = int(lm.x * img_w), int(lm.y * img_h)
                    # Get the 2D Coordinates
                    x_points.append(x)
                    y_points.append(y)
                    
            # convert the list to numpy array to convert it easy to dataframe 
            x_points=np.array(np.transpose(x_points).astype('float'),dtype=np.float32).reshape(1,-1)
            y_points=np.array(np.transpose(y_points).astype('float'),dtype=np.float32).reshape(1,-1)
            dfx2=pd.DataFrame(x_points)
            dfy2=pd.DataFrame(y_points)
        # concating the current points of current image with the previous image points 
        dfx = pd.concat([dfx,dfx2],axis=0)
        dfy = pd.concat([dfy,dfy2],axis=0)

## Checking the final dataframes before saving 

In [ ]:
dfx.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1735 entries, 0 to 0
Columns: 468 entries, 0 to 467
dtypes: float32(468)
memory usage: 3.1 MB


## Giving columns names and saving x points to dataframe and y points to dataframe

In [ ]:
dfx.columns=colsx

In [ ]:
dfy.columns=colsy

In [ ]:
dfx.to_csv('/content/drive/MyDrive/Head Pose Estimation ML Project/Dataset/Data/data_x_468.csv',index=False)
dfy.to_csv('/content/drive/MyDrive/Head Pose Estimation ML Project/Dataset/Data/data_y_468.csv',index=False)

## Concating the x,y dataframes with the name of its image

In [ ]:
df_media = pd.DataFrame(pd.concat([ dfx,dfy ],axis=1))
df_media['image_name'] = imgnolist

In [ ]:
df_media.to_csv('/content/drive/MyDrive/Head Pose Estimation ML Project/Dataset/Data/data_media_id.csv',index=False)

In [ ]:
data68 = pd.read_csv('/content/drive/MyDrive/Head Pose Estimation ML Project/Dataset/Data/data_68_low.csv')

In [ ]:
angles_df=data68.iloc[:,-4:]
angles_df.columns=['yaw','pitch','roll','image_name']
angles_df

,yaw,pitch,roll,image_name
0,1.205376,-0.851643,-0.556451,image03258
1,-0.233745,-0.116700,0.095809,image03247
2,-1.445564,-0.541882,1.070782,image03252
3,0.824710,-0.375663,0.001752,image03237
4,0.062462,0.093373,-0.125822,image03250
...,...,...,...,...
1995,-0.237305,-0.329202,0.184649,image00841
1996,0.352952,0.033367,-0.062870,image00844
1997,0.288579,-0.105130,-0.155294,image00855
1998,0.054460,0.530870,-0.088705,image00854


In [ ]:
angles_df.to_csv('/content/drive/MyDrive/Head Pose Estimation ML Project/Dataset/Data/angles_low.csv',index=False)

In [ ]:
# df_left_joined = pd.merge(df1, df2, how='left', on='col1')
df_media_angles = pd.merge(df_media,angles_df,how='left',on='image_name')

In [ ]:
df_media_angles

,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,...,y463,y464,y465,y466,y467,y468,image_name,yaw,pitch,roll
0,247.0,265.0,248.0,254.0,268.0,265.0,253.0,177.0,253.0,256.0,...,292.0,228.0,231.0,234.0,223.0,219.0,image03195,0.725284,-0.140406,-0.174674
1,220.0,211.0,216.0,205.0,210.0,210.0,213.0,163.0,212.0,211.0,...,281.0,222.0,224.0,226.0,212.0,210.0,image03247,-0.233745,-0.116700,0.095809
2,188.0,171.0,188.0,173.0,168.0,171.0,182.0,174.0,182.0,178.0,...,272.0,219.0,223.0,225.0,209.0,203.0,image03214,-1.330352,0.003072,0.362599
3,223.0,222.0,225.0,216.0,223.0,223.0,227.0,157.0,228.0,228.0,...,297.0,213.0,219.0,224.0,197.0,193.0,image03238,-0.040907,-0.474087,0.019801
4,219.0,227.0,223.0,222.0,229.0,230.0,232.0,165.0,234.0,236.0,...,288.0,218.0,221.0,224.0,213.0,210.0,image03236,0.205732,-0.241359,-0.150232
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1730,225.0,238.0,230.0,236.0,241.0,242.0,243.0,183.0,247.0,250.0,...,287.0,228.0,230.0,232.0,228.0,227.0,image00860,0.218578,-0.140524,-0.294115
1731,193.0,172.0,189.0,172.0,170.0,171.0,181.0,168.0,181.0,178.0,...,282.0,227.0,229.0,231.0,220.0,215.0,image00831,-1.241852,-0.948136,1.234426
1732,217.0,218.0,218.0,215.0,219.0,221.0,224.0,168.0,227.0,228.0,...,281.0,224.0,225.0,227.0,220.0,218.0,image00832,-0.011567,-0.110805,-0.074486
1733,250.0,266.0,254.0,258.0,267.0,266.0,261.0,200.0,262.0,264.0,...,272.0,224.0,226.0,228.0,227.0,224.0,image00861,0.983607,0.236384,-0.383734


In [ ]:
df_media_angles.to_csv('/content/drive/MyDrive/Head Pose Estimation ML Project/Dataset/Data/media_with_angles_low.csv',index=False)